In [264]:
import re
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from torch.utils.data import TensorDataset
# 首先读取用户上传的CSV文件，并检查前几行数据
file_path = '测试2.csv'
df = pd.read_csv(file_path, encoding='utf-8')
df.head()

# 定义一个函数来清理文本
def clean_text(text):
    # 移除表名
    table_pattern = re.compile(r'(\w+\.\w+)')
    # 移除网址
    url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\$\$,]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    # 使用正则表达式去除表名
    table_name_pattern =  re.compile(r'\b[A-Za-z0-9_\u4e00-\u9fa5]+_[A-Za-z0-9_\u4e00-\u9fa5]+(?:-\u4e00-\u9fa5)+\b')
    # 移除人名
  #  name_pattern = re.compile(r'^(?:[\u4e00-\u9fa5]+)(?:·[\u4e00-\u9fa5]+)*$|^[a-zA-Z0-9]+\s?[\.·\-()a-zA-Z]*[a-zA-Z]+$')
    # 移除erp
    pinyin_pattern1 = r'\b[a-z][a-z0-9]*\b'
    pinyin_pattern2 = r'[a-zA-Z]+'
    # 赋值
    text = re.sub('报表ID','',text)
    text = re.sub('报表id','',text)
    text = re.sub('数据集ID','',text)
    text = re.sub('数据集id','',text)
    text = re.sub('udt','',text,flags=re.IGNORECASE)
    text = re.sub('ud','',text,flags=re.IGNORECASE)
    text = re.sub('uda','',text,flags=re.IGNORECASE)
    text = re.sub('UDATE','',text,flags=re.IGNORECASE)
    text = re.sub(r'[a-zA-Z]{4,15}\d{1,3}(?![\d])','',text) # 移除erp
    text = re.sub('用户追加信息如下','',text)
    text = re.sub(r'ID','',text,flags=re.IGNORECASE)
    text = re.sub(r'udata', '', text, flags=re.IGNORECASE)
    text = re.sub(r'erp', '', text, flags=re.IGNORECASE)
    text = re.sub(r'udate', '', text, flags=re.IGNORECASE)
#    text = re.sub('离职','',text)
    text = re.sub('同事','',text)
    text = re.sub('员工','',text)
    text = re.sub('智网','',text)
    text = re.sub('分拣','',text)
    text = re.sub('小胖达','',text)
    text = re.sub('胖达','',text)
    text = re.sub('帮忙','',text)
#    text = re.sub('链接','',text)
    text = re.sub(url_pattern, '', text) # 移除网址
    # 移除sql语句
 #   text = re.sub('\b(SELECT|INSERT\s+INTO|UPDATE|DELETE\s+FROM)\b[\s\S]*?(?=\b(FROM|WHERE|ORDER\s+BY[\s\S]+?BY|GROUP\s+BY[\s\S]+?BY|HAVING|LIMIT[\s\S]+?\d+|OFFSET[\s\S]+?\d+|FETCH|UNION|EXCEPT|INTERSECT|$))','',text)
    text = re.sub(r'([\u4e00-\u9fa5])([a-zA-Z0-9])', r'\1 \2', text) # 中英文加空格
    text = re.sub(r'([a-zA-Z0-9])([\u4e00-\u9fa5])', r'\1 \2', text) 
    text = re.sub(r'([a-zA-Z])([,.!?;:])', r'\1 \2', text)             # 英文后接标点符号
    text = re.sub(r'([a-zA-Z])([0-9])', r'\1 \2', text)                # 英文后接数字
    text = re.sub(r'\b[a-zA-Z0-9]+_[a-zA-Z0-9]+(?:_[a-zA-Z0-9]+)*\b','',text)
    text = re.sub(r'\s+', ' ', text)  # 替换多余的空格为单个空格
    text = re.sub(r'\n', ' ', text)   # 替换换行符为空格
    text = re.sub(table_pattern, '', text)  # 移除英文表名
    text = re.sub(table_name_pattern, '', text)  # 移除中文表名
    text = re.sub(r'\d{4,6}(?![\d])', '', text) # 移除报表id

#    text = re.sub(name_pattern, '', text)  # 移除人名
#    text = re.sub(pinyin_pattern1, '', text)  # 移除erp
 #   text = re.sub(pinyin_pattern2, '', text)  # 移除erp
    # 保留中英文和空格，去除数字
#    text = re.sub(r'[0-9]', '', text)
    return text.strip()

# 清理'df' DataFrame中的'text'列
df['text'] = df['text'].apply(clean_text)

# 检查清理后的数据
df.head()
df.to_csv('中间表.csv', index=False)

In [247]:
# 检查'label'列的唯一值
unique_labels = df['label'].unique()
unique_labels.sort()  # 对标签进行排序以便于查看
unique_labels


array([0, 1], dtype=int64)

In [265]:
from sklearn.model_selection import train_test_split

# 分割数据集为训练集和验证集，test_size=0.3意味着验证集占总数据的30%
train_data, val_data = train_test_split(df, test_size=0.3, random_state=42)

# 仅保留'text'和'label'列
train_data = train_data[['text', 'label']]
val_data = val_data[['text', 'label']]

# 检查训练集和验证集的大小
train_data_size = len(train_data)
val_data_size = len(val_data)

train_data_size, val_data_size

(1381, 592)

In [266]:
import re
import jieba

# 读取停用词文件并转换为集合
def load_stopwords(file_path):
    stopwords = set()
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # 去除每行两端的空白字符，包括换行符
            word = line.strip()
            # 如果单词非空，则添加到停用词集合中
            if word:
                stopwords.add(word)
    return stopwords

# 调用函数，传入停用词文件的路径
stopwords = load_stopwords('停用词.txt')
# 优化后的分词函数，处理中英文，去除数字
def jieba_tokenize(text, stopwords):
    words = jieba.lcut(text)  # 使用 jieba 进行分词
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

# 应用 jieba 分词和去除停用词的函数到训练集和验证集的'text'列
train_data['text'] = train_data['text'].apply(lambda x: jieba_tokenize(x, stopwords))
val_data['text'] = val_data['text'].apply(lambda x: jieba_tokenize(x, stopwords))

# 检查处理后的训练集和验证集
print(train_data.head())
print(val_data.head())
train_data.to_csv('中间表_去除停用词后_训练集.csv', index=False)
val_data.to_csv('中间表_去除停用词后_验证集.csv', index=False)

                                                   text  label
900   京东 秀洲 高照 营业部 涉及   UE 01 YY KD   零售   promise  ...      0
774           表在 中午     月     号 数据 下午   17   点     号 数据      0
1615                     数据 对不上 线下 导出 计算 跟线   编辑 页面 进一步      0
1565                        分组 汇总 如何 想 计算 网点 每月 分子 分母 量      1
1317                                      数据 未 更新 辛苦 原因      0
                                                   text  label
759                                  正式工 出勤 人次 打卡 系统 出入      0
297                                         查看 视频 讲解 声音      0
1421                     ata   怎么 两个   sheet   合并 公式 粘贴      1
350   妥投 率用 实时 运单 监控 小时 快照 fdm   张表 底表 研发 底表 添加   K ...      0
1103                            显示 筛选 框 自定义 名称 重复 未见 重复      1


In [226]:
from collections import Counter

# 合并训练集和验证集以便统一计算词频
all_data = pd.concat([train_data, val_data])

# 1. 计算每个词的出现频率
word_counts = Counter()
for words in all_data['text'].str.split():
    word_counts.update(words)

# 2. 设定最低词频阈值
min_word_freq = 2

# 3. 保留那些出现频率大于或等于阈值的词
words_to_keep = {word for word, count in word_counts.items() if count >= min_word_freq}

# 4. 定义一个函数来去除稀有词
def remove_rare_words(text, words_to_keep):
    words = text.split()
    filtered_words = [word for word in words if word in words_to_keep]
    return ' '.join(filtered_words)

# 5. 应用去除稀有词的函数到训练集和验证集的'text'列
train_data['text'] = train_data['text'].apply(lambda x: remove_rare_words(x, words_to_keep))
val_data['text'] = val_data['text'].apply(lambda x: remove_rare_words(x, words_to_keep))

# 6. 检查处理后的训练集和验证集
train_data.head(), val_data.head()

train_data.to_csv('中间表_去除停用词后_训练集.csv', index=False)
val_data.to_csv('中间表_去除停用词后_验证集.csv', index=False)

In [267]:
# 过滤掉token数量小于等于1的行
train_data = train_data[train_data['text'].str.split().str.len() > 1]
val_data = val_data[val_data['text'].str.split().str.len() > 1]

train_data.to_csv('中间表_去除停用词后_训练集.csv', index=False)
val_data.to_csv('中间表_去除停用词后_验证集.csv', index=False)

In [6]:
# 将label中的15转换为1，其他转换为0
train_data['label'] = train_data['label'].apply(lambda x: 1 if x == 15 else 0)
val_data['label'] = val_data['label'].apply(lambda x: 1 if x == 15 else 0)
# 输出处理后的前五行数据
train_data.head()

text  label
1371                                     推送 数据 提示 图片 异常      0
126   旧 报表 删除 然后 上传 excel 想 删掉 提醒 报表 使用 不能 删除 使用 那个 ...      0
327             相同 数据 准备 里面 bdp 出来 结果 不 一致 怀疑 字 段 空格 导致      0
292                                            查询 不到 数据      0
1058                                  邮件 推送 无法 保存 邮件 内容      0

In [268]:
from sklearn.utils import resample

train_data = train_data.sample(frac=1, random_state=42)
val_data = val_data.sample(frac=1,random_state=42)
# 欠采样训练集
train_data_1 = train_data[train_data['label'] == 1]
train_data_0 = train_data[train_data['label'] == 0]
train_data_0_downsampled = resample(train_data_0, 
                                    replace=False, 
                                    n_samples=len(train_data_1), 
                                    random_state=42)
train_data_downsampled = pd.concat([train_data_0_downsampled, train_data_1])

# 欠采样验证集
val_data_1 = val_data[val_data['label'] == 1]
val_data_0 = val_data[val_data['label'] == 0]
val_data_0_downsampled = resample(val_data_0, 
                                  replace=False, 
                                  n_samples=len(val_data_1), 
                                  random_state=42)
val_data_downsampled = pd.concat([val_data_0_downsampled, val_data_1])

# 现在 train_data_downsampled 和 val_data_downsampled 是平衡的数据集


# 十五分类

In [25]:

# 检查分割后的训练集和验证集的大小
train_data_10th_size = len(train_data_10th)
val_data_10th_size = len(val_data_10th)
print(f'训练集大小: {train_data_10th_size}, 验证集大小: {val_data_10th_size}')

# 初始化分词器和BERT模型
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=15)  # 15个分类

# 编码文本
train_encoding = tokenizer(train_data_10th['text'].tolist(), padding=True, truncation=True, return_tensors='pt')
val_encoding = tokenizer(val_data_10th['text'].tolist(), padding=True, truncation=True, return_tensors='pt')

# 创建自定义数据集
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx] - 1)  # 减去1以匹配模型的索引
        return item

    def __len__(self):
        return len(self.labels)

# 准备数据集
train_labels = torch.tensor(train_data_10th['label'].tolist())
train_dataset = TextDataset(train_encoding, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)

val_labels = torch.tensor(val_data_10th['label'].tolist())
val_dataset = TextDataset(val_encoding, val_labels)
val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=False)

# 准备优化器
optimizer = AdamW(model.parameters(), lr=5e-5)

# 训练模型
model.train()
for epoch in range(3):  # 训练3个epochs
    true_labels = []
    predicted_labels = []
    for batch in train_dataloader:
        inputs = {key: val for key, val in batch.items() if key != 'labels'}
        outputs = model(**inputs, labels=batch['labels'])
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        # 获取预测结果
        logits = outputs.logits
        predicted = torch.argmax(logits, dim=1)
        true_labels.extend(batch['labels'].tolist())
        predicted_labels.extend(predicted.tolist())

    # 计算准确率
        train_accuracy = accuracy_score(true_labels, predicted_labels)
    print(f'Epoch {epoch + 1}/{3} - Train Loss: {loss.item()} - Train Accuracy: {train_accuracy}')

    # 评估模型
    model.eval()
    true_labels = []
    predicted_labels = []
    with torch.no_grad():
        for batch in val_dataloader:
            inputs = {key: val for key, val in batch.items() if key != 'labels'}
            outputs = model(**inputs)
            logits = outputs.logits
            predicted = torch.argmax(logits, dim=1)
            true_labels.extend(batch['labels'].tolist())
            predicted_labels.extend(predicted.tolist())

    # 计算准确率
    val_accuracy = accuracy_score(true_labels, predicted_labels)
    print(f'Epoch {epoch + 1}/{3} - Val Loss: {loss.item()} - Val Accuracy: {val_accuracy}')


训练集大小: 213, 验证集大小: 92


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


IndexError: Target -1 is out of bounds.

# 二分类

In [269]:

# 假设train_data_10th和val_data_10th已经按照之前的指示处理过label列

# 检查分割后的训练集和验证集的大小
train_data_10th_size = len(train_data_downsampled)
val_data_10th_size = len(val_data_downsampled)
print(f'训练集大小: {train_data_10th_size}, 验证集大小: {val_data_10th_size}')

# 初始化分词器和BERT模型
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=2)  # 二分类

# 编码文本
train_encoding = tokenizer(train_data_downsampled['text'].tolist(), padding=True, truncation=True, return_tensors='pt')
val_encoding = tokenizer(val_data_downsampled['text'].tolist(), padding=True, truncation=True, return_tensors='pt')

# 创建自定义数据集
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])  # 直接使用标签
        return item

    def __len__(self):
        return len(self.labels)

# 准备数据集
train_labels = torch.tensor(train_data_downsampled['label'].tolist())
train_dataset = TextDataset(train_encoding, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)

val_labels = torch.tensor(val_data_downsampled['label'].tolist())
val_dataset = TextDataset(val_encoding, val_labels)
val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=False)

# 准备优化器
optimizer = AdamW(model.parameters(), lr=5e-5)

# 训练模型
model.train()
for epoch in range(3):  # 训练3个epochs
    true_labels = []
    predicted_labels = []
    for batch in train_dataloader:
        inputs = {key: val for key, val in batch.items() if key != 'labels'}
        outputs = model(**inputs, labels=batch['labels'])
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        # 获取预测结果
        logits = outputs.logits
        predicted = torch.argmax(logits, dim=1)
        true_labels.extend(batch['labels'].tolist())
        predicted_labels.extend(predicted.tolist())

    # 计算准确率
    train_accuracy = accuracy_score(true_labels, predicted_labels)
    print(f'Epoch {epoch + 1}/{3} - Train Loss: {loss.item()} - Train Accuracy: {train_accuracy}')

    # 评估模型
    model.eval()
    true_labels = []
    predicted_labels = []
    with torch.no_grad():
        for batch in val_dataloader:
            inputs = {key: val for key, val in batch.items() if key != 'labels'}
            outputs = model(**inputs)
            logits = outputs.logits
            predicted = torch.argmax(logits, dim=1)
            true_labels.extend(batch['labels'].tolist())
            predicted_labels.extend(predicted.tolist())

    # 计算准确率
    val_accuracy = accuracy_score(true_labels, predicted_labels)
    print(f'Epoch {epoch + 1}/{3} - Val Accuracy: {val_accuracy}')


训练集大小: 1148, 验证集大小: 522


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3 - Train Loss: 1.0910680294036865 - Train Accuracy: 0.6550522648083623
Epoch 1/3 - Val Accuracy: 0.6704980842911877
Epoch 2/3 - Train Loss: 0.6250318884849548 - Train Accuracy: 0.6907665505226481
Epoch 2/3 - Val Accuracy: 0.5862068965517241
Epoch 3/3 - Train Loss: 2.1223831176757812 - Train Accuracy: 0.730836236933798
Epoch 3/3 - Val Accuracy: 0.6762452107279694
